<a href="https://colab.research.google.com/github/kabilan942/Legal/blob/main/LDS_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers
!pip install sentencepiece
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM 
import nltk
!pip install rouge
nltk.download('punkt')
from rouge import Rouge

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 97.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 120.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 48.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [3]:
import os

documentPath = f'/content/drive/MyDrive/Legal Data/Indian Legal Abs Ext/dataset/IN-Abs/test-data/judgement/'
dir = os.listdir(documentPath)

dir_list = []
for i in range(len(dir)):
  dir_list.append(int(dir[i][:-4]))

In [ ]:
"""
def read_input():
    file = open("D:/Phd_work/Dataset/Legal dataset/dataset/IN-Abs/train-data/judgement/6.txt", "r")
    FileContent = file.read().strip() # Stripping the white spaces in the file
    file = open("D:/Phd_work/Dataset/Legal dataset/dataset/IN-Abs/train-data/summary/6.txt", "r")
    summaryContent = file.read().strip()
    return(FileContent,summaryContent)
"""

# BART - Chunking (Single Document)

In [ ]:
def read_input():
  doc_name = '/content/drive/MyDrive/Legal Data/Indian Legal Abs Ext/dataset/IN-Abs/test-data/judgement/78.txt'
  doc = open(doc_name,'r')
  FileContent = doc.read().strip()
  sum_name ='/content/drive/MyDrive/Legal Data/Indian Legal Abs Ext/dataset/IN-Abs/test-data/summary/78.txt'
  sum = open(sum_name,'r')
  summaryContent = sum.read().strip()
  return(FileContent,summaryContent)


In [ ]:
def model_load():
    checkpoint = "sshleifer/distilbart-cnn-12-6" #model loaded from Huggingface
    tokenizer = AutoTokenizer.from_pretrained(checkpoint)
    model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)
#Model Details 
    print("tokenizer.model_max_length :",tokenizer.model_max_length )# max token inputted in model 1024
    print(" max tokens excluding the special tokens",tokenizer.max_len_single_sentence )# max tokens for input in model 1022
    print("num_special_tokens_to_add ",tokenizer.num_special_tokens_to_add() ) # special token needed by model 2
    return(tokenizer,model)


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 73.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 125.9 MB/s eta 0:00:00


In [ ]:
def chunking(FileContent,tokenizer):
    # extract the sentences from the document
    sentences = nltk.tokenize.sent_tokenize(FileContent) #split the data(paragraph) into individual sentences
    print(len(sentences)) #print total no of sentences
    # find the max tokens in the longest sentence
    maxtokencount=max([len(tokenizer.tokenize(sentence)) for sentence in sentences])
    print("maximum tokens in longest sentence:",maxtokencount)

    # initialize
    length = 0
    chunk = ""
    chunks = []
    count = -1
    for sentence in sentences:
        count += 1
        combined_length = len(tokenizer.tokenize(sentence)) + length # add the no. of sentence tokens to the length counter
        #print(combined_length)  #combined length of all the tokens in all sentences 

        if combined_length  <= tokenizer.max_len_single_sentence: # if it doesn't exceed than max_len(1022)
            chunk += sentence + " " # add the sentence to the chunk with white space
            length = combined_length # update the length counter

            # if it is the last sentence
            if count == len(sentences) - 1:
                chunks.append(chunk.strip()) # save the chunk
    
        else:   # if max token limit reaches then save the chunk
            chunks.append(chunk.strip()) # save the chunk
    
        # reset the parameter to add next tokens to next chunk
            length = 0 
            chunk = ""

        # take care of the overflow sentence
            chunk += sentence + " "
            length = len(tokenizer.tokenize(sentence))

            
    print("Total no of chunks created",len(chunks))
    print("Total no of tokens generated by chunks",sum([len(tokenizer(c).input_ids) for c in chunks]))
    print("Total no of token in original text",len(tokenizer(FileContent).input_ids))
    print("No of tokens in each  chunk excluding the special token",[len(tokenizer.tokenize(c)) for c in chunks])
    return (chunks)

In [ ]:
def sum_generate(chunks,tokenizer,model):
    final_summ_gen=""
    inputs = [tokenizer(chunk, return_tensors="pt") for chunk in chunks] #genrating inputs with chunks pt stands for pytorch
    for input in inputs:
        output = model.generate(**input)    #output generated for each chunk
        summ_generated=tokenizer.decode(*output, skip_special_tokens=True)# decoding the output
        final_summ_gen= final_summ_gen+summ_generated
    print(final_summ_gen)
    return(final_summ_gen)

In [ ]:
def rouge_score(summaryContent,final_summ_gen):  
    rouge = Rouge()
    result_rouge=rouge.get_scores(summaryContent, final_summ_gen, avg=True)
# Printing the final list
    print(result_rouge)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
# inputs to the model
if __name__ == "__main__":
  FileContent,summaryContent=read_input()
  tokenizer,model=model_load()
  chunks=chunking(FileContent,tokenizer)
  final_summ_gen=sum_generate(chunks,tokenizer,model)
  result_rouge=rouge_score(summaryContent,final_summ_gen)

Token indices sequence length is longer than the specified maximum sequence length for this model (2634 > 1024). Running this sequence through the model will result in indexing errors


tokenizer.model_max_length : 1024
 max tokens excluding the special tokens 1022
num_special_tokens_to_add  2
98
maximum tokens in longest sentence: 137
Total no of chunks created 3
Total no of tokens generated by chunks 2572
Total no of token in original text 2634
No of tokens in each  chunk excluding the special token [999, 1019, 548]


/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 142 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


 Appeal from an Order of the High Court of Bombay (Bavdekar and Chainani JJ.) dated 20th February, 1950, in Criminal Appeal No. 106 of 1950. W.H. King was tenant of a flat on the second floor of a building called "Ganga Vihar", Marine Drive, Bombay, which belongs to a lady named Durgeshwari Devi. The wife of the appellant denied any aiding and abetting. The wife was acquitted, the evidence being insufficient to prove any abetment. The appellant preferred an appeal to the High Court of Bombay but it was summarily dismissed. He asked for a certificate under article 134(1)(c) of the Constitution but this was rejected on 10 4 1950. He applied for special leave to appeal to this Court and it was granted on 3 10 1950. The word "relinquishment" does not occur in the Transfer of Property Act but it is found in many of the Tenancy Acts in various provinces where there are Sec tions which deal with the relinquishment of their holdings by tenants in favour of the landlord by notice given to him i

In [ ]:
print(final_summ_gen)

 Appeal from an Order of the High Court of Bombay (Bavdekar and Chainani JJ.) dated 20th February, 1950, in Criminal Appeal No. 106 of 1950. W.H. King was tenant of a flat on the second floor of a building called "Ganga Vihar", Marine Drive, Bombay, which belongs to a lady named Durgeshwari Devi. The wife of the appellant denied any aiding and abetting. The wife was acquitted, the evidence being insufficient to prove any abetment. The appellant preferred an appeal to the High Court of Bombay but it was summarily dismissed. He asked for a certificate under article 134(1)(c) of the Constitution but this was rejected on 10 4 1950. He applied for special leave to appeal to this Court and it was granted on 3 10 1950. The word "relinquishment" does not occur in the Transfer of Property Act but it is found in many of the Tenancy Acts in various provinces where there are Sec tions which deal with the relinquishment of their holdings by tenants in favour of the landlord by notice given to him i

In [ ]:
print("Given Summary:",summaryContent)

Given Summary: Sub section (1) of sec.
19 of the Bombay Rents, Hotel and Lodging House Rates Control Act, LVI I of 1947, provided that "it shall not be lawful for the tenant or any person acting or purporting to act on behalf of the tenant to claim or receive any sum or any consideration as a condition for the relinquishment of his tenancy of any premises"; and sub sec.
(2) provided that any tenant or person who in contravention of the provisions of sub sec.
(1) receives any sum or consideration shall on conviction be punished with imprisonment and also with fine.
A. who was a tenant of a flat, handed over vacant posses sion the flat to B on receiving "pugree", under a document which recited that A shall have no claim whatever over the flat and that B shall pay the rent directly to the landlord.
A was convicted of an offence under sec.
19 (2).
Held, that there was no "relinquishment" of his tenancy by A, within the meaning of sec.
19 (1) and the conviction could not be sustained.
There

tokenizer.model_max_length : 1024

max tokens excluding the special tokens 1022

num_special_tokens_to_add  2  98

maximum tokens in longest sentence: 137

Total no of chunks created 3

Total no of tokens generated by chunks 2572

Total no of token in original text 2634

No of tokens in each  chunk excluding the special token [999, 1019, 548]



{'rouge-1': {'r': 0.291044776119403, 'p': 0.20855614973262032, 'f': 0.24299064934191253}, 

'rouge-2': {'r': 0.08673469387755102, 'p': 0.049132947976878616, 'f': 0.06273062268923388}, 

'rouge-l': {'r': 0.26119402985074625, 'p': 0.18716577540106952, 'f': 0.21806853096185025}}

# BART - Chunking (Multiple Documents)

In [4]:
def read_input(i):
  doc_name = f'/content/drive/MyDrive/Legal Data/Indian Legal Abs Ext/dataset/IN-Abs/test-data/judgement/{i}.txt'
  doc = open(doc_name,'r')
  FileContent = doc.read().strip()
  sum_name =f'/content/drive/MyDrive/Legal Data/Indian Legal Abs Ext/dataset/IN-Abs/test-data/summary/{i}.txt'
  sum = open(sum_name,'r')
  summaryContent = sum.read().strip()
  return(FileContent,summaryContent)


In [5]:
def model_load():
    checkpoint = "sshleifer/distilbart-cnn-12-6" #model loaded from Huggingface
    tokenizer = AutoTokenizer.from_pretrained(checkpoint)
    model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)
#Model Details 
    #print("tokenizer.model_max_length :",tokenizer.model_max_length )# max token inputted in model 1024
    #print(" max tokens excluding the special tokens",tokenizer.max_len_single_sentence )# max tokens for input in model 1022
    #print("num_special_tokens_to_add ",tokenizer.num_special_tokens_to_add() ) # special token needed by model 2
    return(tokenizer,model)


In [6]:
def chunking(FileContent,tokenizer):
    # extract the sentences from the document
    sentences = nltk.tokenize.sent_tokenize(FileContent) #split the data(paragraph) into individual sentences
    #print(len(sentences)) #print total no of sentences
    # find the max tokens in the longest sentence
    maxtokencount=max([len(tokenizer.tokenize(sentence)) for sentence in sentences])
    #print("maximum tokens in longest sentence:",maxtokencount)

    # initialize
    length = 0
    chunk = ""
    chunks = []
    count = -1
    for sentence in sentences:
        count += 1
        combined_length = len(tokenizer.tokenize(sentence)) + length # add the no. of sentence tokens to the length counter
        #print(combined_length)  #combined length of all the tokens in all sentences 

        if combined_length  <= tokenizer.max_len_single_sentence: # if it doesn't exceed than max_len(1022)
            chunk += sentence + " " # add the sentence to the chunk with white space
            length = combined_length # update the length counter

            # if it is the last sentence
            if count == len(sentences) - 1:
                chunks.append(chunk.strip()) # save the chunk
    
        else:   # if max token limit reaches then save the chunk
            chunks.append(chunk.strip()) # save the chunk
    
        # reset the parameter to add next tokens to next chunk
            length = 0 
            chunk = ""

        # take care of the overflow sentence
            chunk += sentence + " "
            length = len(tokenizer.tokenize(sentence))

            
    #print("Total no of chunks created",len(chunks))
    #print("Total no of tokens generated by chunks",sum([len(tokenizer(c).input_ids) for c in chunks]))
    #print("Total no of token in original text",len(tokenizer(FileContent).input_ids))
    #print("No of tokens in each  chunk excluding the special token",[len(tokenizer.tokenize(c)) for c in chunks])
    return (chunks)

In [7]:
def sum_generate(chunks,tokenizer,model):
    final_summ_gen=""
    inputs = [tokenizer(chunk, return_tensors="pt") for chunk in chunks] #genrating inputs with chunks pt stands for pytorch
    for input in inputs:
        output = model.generate(**input)    #output generated for each chunk
        summ_generated=tokenizer.decode(*output, skip_special_tokens=True)# decoding the output
        final_summ_gen= final_summ_gen+summ_generated
    #print(final_summ_gen)
    return(final_summ_gen)

In [8]:
def rouge_score(summaryContent,final_summ_gen):  
    rouge = Rouge()
    result_rouge=rouge.get_scores(summaryContent, final_summ_gen, avg=True)
# Printing the final list
    return result_rouge

In [9]:
# inputs to the model
if __name__ == "__main__":
  result_rouge_list = []
  for i in dir_list[:30]:
    FileContent,summaryContent=read_input(i)
    tokenizer,model=model_load()
    chunks=chunking(FileContent,tokenizer)
    final_summ_gen=sum_generate(chunks,tokenizer,model)
    result_rouge_list.append(rouge_score(summaryContent,final_summ_gen))

/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 142 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [10]:
len(result_rouge_list)

30

In [11]:
agg_rouge_score = {'rouge-1':{'r':0,'p':0,'f':0},'rouge-2':{'r':0,'p':0,'f':0},
                   'rouge-l':{'r':0,'p':0,'f':0}}

for i in range(len(result_rouge_list)):
  for rouge_type in ['rouge-1','rouge-2','rouge-l']:
    for j in ['r','p','f']:
      agg_rouge_score[rouge_type][j]+=(result_rouge_list[i][rouge_type][j]/len(result_rouge_list))

print(agg_rouge_score)

{'rouge-1': {'r': 0.5146350977636011, 'p': 0.2939724380970604, 'f': 0.3647993139364114}, 'rouge-2': {'r': 0.293680200989159, 'p': 0.1405336410499587, 'f': 0.18339711797892372}, 'rouge-l': {'r': 0.4749069546202089, 'p': 0.26922998613889076, 'f': 0.3350146321923683}}


# BigBird - Chunking (Single Document)

In [ ]:
def read_input():
  doc_name = '/content/drive/MyDrive/Legal Data/Indian Legal Abs Ext/dataset/IN-Abs/test-data/judgement/78.txt'
  doc = open(doc_name,'r')
  FileContent = doc.read().strip()
  sum_name ='/content/drive/MyDrive/Legal Data/Indian Legal Abs Ext/dataset/IN-Abs/test-data/summary/78.txt'
  sum = open(sum_name,'r')
  summaryContent = sum.read().strip()
  return(FileContent,summaryContent)


In [ ]:
from transformers import BigBirdForCausalLM

def model_load():
    checkpoint = "google/bigbird-roberta-base" #model loaded from Huggingface
    tokenizer = AutoTokenizer.from_pretrained(checkpoint)
    model = BigBirdForCausalLM.from_pretrained(checkpoint,is_decoder=True)
#Model Details 
    print("tokenizer.model_max_length :",tokenizer.model_max_length )# max token inputted in model 1024
    print(" max tokens excluding the special tokens",tokenizer.max_len_single_sentence )# max tokens for input in model 1022
    print("num_special_tokens_to_add ",tokenizer.num_special_tokens_to_add() ) # special token needed by model 2
    return(tokenizer,model)


In [ ]:
def chunking(FileContent,tokenizer):
    # extract the sentences from the document
    sentences = nltk.tokenize.sent_tokenize(FileContent) #split the data(paragraph) into individual sentences
    print(len(sentences)) #print total no of sentences
    # find the max tokens in the longest sentence
    maxtokencount=max([len(tokenizer.tokenize(sentence)) for sentence in sentences])
    print("maximum tokens in longest sentence:",maxtokencount)

    # initialize
    length = 0
    chunk = ""
    chunks = []
    count = -1
    for sentence in sentences:
        count += 1
        combined_length = len(tokenizer.tokenize(sentence)) + length # add the no. of sentence tokens to the length counter
        #print(combined_length)  #combined length of all the tokens in all sentences 

        if combined_length  <= tokenizer.max_len_single_sentence: # if it doesn't exceed than max_len(1022)
            chunk += sentence + " " # add the sentence to the chunk with white space
            length = combined_length # update the length counter

            # if it is the last sentence
            if count == len(sentences) - 1:
                chunks.append(chunk.strip()) # save the chunk
    
        else:   # if max token limit reaches then save the chunk
            chunks.append(chunk.strip()) # save the chunk
    
        # reset the parameter to add next tokens to next chunk
            length = 0 
            chunk = ""

        # take care of the overflow sentence
            chunk += sentence + " "
            length = len(tokenizer.tokenize(sentence))

            
    print("Total no of chunks created",len(chunks))
    print("Total no of tokens generated by chunks",sum([len(tokenizer(c).input_ids) for c in chunks]))
    print("Total no of token in original text",len(tokenizer(FileContent).input_ids))
    print("No of tokens in each  chunk excluding the special token",[len(tokenizer.tokenize(c)) for c in chunks])
    return (chunks)

In [ ]:
def sum_generate(chunks,tokenizer,model):
    final_summ_gen=""
    inputs = [tokenizer(chunk, return_tensors="pt") for chunk in chunks] #genrating inputs with chunks pt stands for pytorch
    for input in inputs:
        output = model.generate(**input)    #output generated for each chunk
        summ_generated=tokenizer.decode(*output, skip_special_tokens=True)# decoding the output
        final_summ_gen= final_summ_gen+summ_generated
    print(final_summ_gen)
    return(final_summ_gen)

In [ ]:
from rouge import Rouge
def rouge_score(summaryContent,final_summ_gen):  
    rouge = Rouge()
    result_rouge=rouge.get_scores(summaryContent, final_summ_gen, avg=True)
# Printing the final list
    print(result_rouge)

In [ ]:
# inputs to the model
if __name__ == "__main__":
  FileContent,summaryContent=read_input()
  tokenizer,model=model_load()
  chunks=chunking(FileContent,tokenizer)
  final_summ_gen=sum_generate(chunks,tokenizer,model)
  result_rouge=rouge_score(summaryContent,final_summ_gen)

Some weights of the model checkpoint at google/bigbird-roberta-base were not used when initializing BigBirdForCausalLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BigBirdForCausalLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BigBirdForCausalLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we reco

tokenizer.model_max_length : 4096
 max tokens excluding the special tokens 4094
num_special_tokens_to_add  2
98
maximum tokens in longest sentence: 138
Total no of chunks created 1
Total no of tokens generated by chunks 2571
Total no of token in original text 2636
No of tokens in each  chunk excluding the special token [2569]
N: Criminal Appeal No. 8 of 1951. Appeal from an Order of the High Court of Bombay (Bavdekar and Chainani JJ.) dated 20th February, 1950, in Criminal Appeal No. 106 of 1950 arising out of an order dated 9th January, 1950, of the Presidency Magistrate, 19th Court, Esplanade, Bombay, in Case No. 10879/P of 1949. The facts are stated in the judgment. Iswarlal C. Dalai and R.B. Dalai, for the appellant. C.K. Daphtary, Solicitor General for India (G. N. Joshi, with him) for the Republic of India (respondent No. 1).Jindra Lal for the respondent No. 2. 1952. February 1. The Judgment of the Court was deliv ered by CHANDRASEKHARA AIYAR J. The facts out of which this Crimi 

If you want to use `BigBirdForCausalLM` as a standalone, add `is_decoder=True.`

tokenizer.model_max_length : 4096

max tokens excluding the special tokens 4094

num_special_tokens_to_add  2  98

maximum tokens in longest sentence: 138

Total no of chunks created 1

Total no of tokens generated by chunks 2571

Total no of token in original text 2636

No of tokens in each  chunk excluding the special token [2569]

/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
**Input length of input_ids is 2571, but `max_length` is set to 20. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.**

{'rouge-1': {'r': 0.22176308539944903, 'p': 0.8609625668449198, 'f': 0.35268345785982924}, 

'rouge-2': {'r': 0.15072830905636478, 'p': 0.6878612716763006, 'f': 0.2472727243240533}, 

'rouge-l': {'r': 0.22038567493112948, 'p': 0.8556149732620321, 'f': 0.35049287735599577}}

# BigBird - Chunking (Multiple Documents)

In [12]:
def read_input(i):
  doc_name = f'/content/drive/MyDrive/Legal Data/Indian Legal Abs Ext/dataset/IN-Abs/test-data/judgement/{i}.txt'
  doc = open(doc_name,'r')
  FileContent = doc.read().strip()
  sum_name =f'/content/drive/MyDrive/Legal Data/Indian Legal Abs Ext/dataset/IN-Abs/test-data/summary/{i}.txt'
  sum = open(sum_name,'r')
  summaryContent = sum.read().strip()
  return(FileContent,summaryContent)


In [18]:
from transformers import BigBirdForCausalLM

def model_load():
    checkpoint = "google/bigbird-roberta-base" #model loaded from Huggingface
    tokenizer = AutoTokenizer.from_pretrained(checkpoint)
    model = BigBirdForCausalLM.from_pretrained(checkpoint,is_decoder=True)
#Model Details 
    #print("tokenizer.model_max_length :",tokenizer.model_max_length )# max token inputted in model 1024
    #print(" max tokens excluding the special tokens",tokenizer.max_len_single_sentence )# max tokens for input in model 1022
    #print("num_special_tokens_to_add ",tokenizer.num_special_tokens_to_add() ) # special token needed by model 2
    return(tokenizer,model)


In [14]:
def chunking(FileContent,tokenizer):
    # extract the sentences from the document
    sentences = nltk.tokenize.sent_tokenize(FileContent) #split the data(paragraph) into individual sentences
    #print(len(sentences)) #print total no of sentences
    # find the max tokens in the longest sentence
    maxtokencount=max([len(tokenizer.tokenize(sentence)) for sentence in sentences])
    #print("maximum tokens in longest sentence:",maxtokencount)

    # initialize
    length = 0
    chunk = ""
    chunks = []
    count = -1
    for sentence in sentences:
        count += 1
        combined_length = len(tokenizer.tokenize(sentence)) + length # add the no. of sentence tokens to the length counter
        #print(combined_length)  #combined length of all the tokens in all sentences 

        if combined_length  <= tokenizer.max_len_single_sentence: # if it doesn't exceed than max_len(1022)
            chunk += sentence + " " # add the sentence to the chunk with white space
            length = combined_length # update the length counter

            # if it is the last sentence
            if count == len(sentences) - 1:
                chunks.append(chunk.strip()) # save the chunk
    
        else:   # if max token limit reaches then save the chunk
            chunks.append(chunk.strip()) # save the chunk
    
        # reset the parameter to add next tokens to next chunk
            length = 0 
            chunk = ""

        # take care of the overflow sentence
            chunk += sentence + " "
            length = len(tokenizer.tokenize(sentence))

            
    #print("Total no of chunks created",len(chunks))
    #print("Total no of tokens generated by chunks",sum([len(tokenizer(c).input_ids) for c in chunks]))
    #print("Total no of token in original text",len(tokenizer(FileContent).input_ids))
    #print("No of tokens in each  chunk excluding the special token",[len(tokenizer.tokenize(c)) for c in chunks])
    return (chunks)

In [15]:
def sum_generate(chunks,tokenizer,model):
    final_summ_gen=""
    inputs = [tokenizer(chunk, return_tensors="pt") for chunk in chunks] #genrating inputs with chunks pt stands for pytorch
    for input in inputs:
        output = model.generate(**input)    #output generated for each chunk
        summ_generated=tokenizer.decode(*output, skip_special_tokens=True)# decoding the output
        final_summ_gen= final_summ_gen+summ_generated
    #print(final_summ_gen)
    return(final_summ_gen)

In [16]:
def rouge_score(summaryContent,final_summ_gen):  
    rouge = Rouge()
    result_rouge=rouge.get_scores(summaryContent, final_summ_gen, avg=True)
# Printing the final list
    return result_rouge

In [19]:
# inputs to the model
if __name__ == "__main__":
  result_rouge_list = []
  for i in dir_list[:30]:
    FileContent,summaryContent=read_input(i)
    tokenizer,model=model_load()
    chunks=chunking(FileContent,tokenizer)
    final_summ_gen=sum_generate(chunks,tokenizer,model)
    result_rouge_list.append(rouge_score(summaryContent,final_summ_gen))

Some weights of the model checkpoint at google/bigbird-roberta-base were not used when initializing BigBirdForCausalLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BigBirdForCausalLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BigBirdForCausalLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we reco

In [22]:
len(result_rouge_list)

30

In [23]:
agg_rouge_score = {'rouge-1':{'r':0,'p':0,'f':0},'rouge-2':{'r':0,'p':0,'f':0},
                   'rouge-l':{'r':0,'p':0,'f':0}}

for i in range(len(result_rouge_list)):
  for rouge_type in ['rouge-1','rouge-2','rouge-l']:
    for j in ['r','p','f']:
      agg_rouge_score[rouge_type][j]+=(result_rouge_list[i][rouge_type][j]/len(result_rouge_list))

print(agg_rouge_score)

{'rouge-1': {'r': 0.29704686623815646, 'p': 0.8199397745193063, 'f': 0.4288183742015664}, 'rouge-2': {'r': 0.18563868603051248, 'p': 0.651633184874414, 'f': 0.28231233567068204}, 'rouge-l': {'r': 0.28884331556801657, 'p': 0.7956824084144307, 'f': 0.41679113893456854}}


# BERT - Chunking (Single Document)

In [ ]:
def read_input():
  doc_name = '/content/drive/MyDrive/Legal Data/Indian Legal Abs Ext/dataset/IN-Abs/test-data/judgement/78.txt'
  doc = open(doc_name,'r')
  FileContent = doc.read().strip()
  sum_name ='/content/drive/MyDrive/Legal Data/Indian Legal Abs Ext/dataset/IN-Abs/test-data/summary/78.txt'
  sum = open(sum_name,'r')
  summaryContent = sum.read().strip()
  return(FileContent,summaryContent)


In [ ]:
from transformers import BertTokenizerFast
from transformers import EncoderDecoderModel

def model_load():
    checkpoint = "bert-base-uncased" #model loaded from Huggingface
    tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")
    tokenizer.bos_token = tokenizer.cls_token
    tokenizer.eos_token = tokenizer.sep_token
    model = EncoderDecoderModel.from_encoder_decoder_pretrained("bert-base-uncased", "bert-base-uncased")
    model.config.vocab_size = model.config.decoder.vocab_size
    model.config.max_length = 142
    model.config.min_length = 56
    model.config.no_repeat_ngram_size = 3
    model.config.early_stopping = True
    model.config.length_penalty = 2.0
    model.config.num_beams = 4
#Model Details 
    print("tokenizer.model_max_length :",tokenizer.model_max_length )# max token inputted in model 1024
    print(" max tokens excluding the special tokens",tokenizer.max_len_single_sentence )# max tokens for input in model 1022
    print("num_special_tokens_to_add ",tokenizer.num_special_tokens_to_add() ) # special token needed by model 2
    return(tokenizer,model)


In [ ]:
import nltk
#nltk.download('punkt')
def chunking(FileContent,tokenizer):
    # extract the sentences from the document
    sentences = nltk.tokenize.sent_tokenize(FileContent) #split the data(paragraph) into individual sentences
    print(len(sentences)) #print total no of sentences
    # find the max tokens in the longest sentence
    maxtokencount=max([len(tokenizer.tokenize(sentence)) for sentence in sentences])
    print("maximum tokens in longest sentence:",maxtokencount)

    # initialize
    length = 0
    chunk = ""
    chunks = []
    count = -1
    for sentence in sentences:
        count += 1
        combined_length = len(tokenizer.tokenize(sentence)) + length # add the no. of sentence tokens to the length counter
        #print(combined_length)  #combined length of all the tokens in all sentences 

        if combined_length  <= tokenizer.max_len_single_sentence: # if it doesn't exceed than max_len(1022)
            chunk += sentence + " " # add the sentence to the chunk with white space
            length = combined_length # update the length counter

            # if it is the last sentence
            if count == len(sentences) - 1:
                chunks.append(chunk.strip()) # save the chunk
    
        else:   # if max token limit reaches then save the chunk
            chunks.append(chunk.strip()) # save the chunk
    
        # reset the parameter to add next tokens to next chunk
            length = 0 
            chunk = ""

        # take care of the overflow sentence
            chunk += sentence + " "
            length = len(tokenizer.tokenize(sentence))

            
    print("Total no of chunks created",len(chunks))
    print("Total no of tokens generated by chunks",sum([len(tokenizer(c).input_ids) for c in chunks]))
    print("Total no of token in original text",len(tokenizer(FileContent).input_ids))
    print("No of tokens in each  chunk excluding the special token",[len(tokenizer.tokenize(c)) for c in chunks])
    return (chunks)

In [ ]:
def sum_generate(chunks,tokenizer,model):
    final_summ_gen=""
    inputs = [tokenizer(chunk, return_tensors="pt") for chunk in chunks] #genrating inputs with chunks pt stands for pytorch
    for input in inputs:
        output = model.generate(**input)    #output generated for each chunk
        summ_generated=tokenizer.decode(*output, skip_special_tokens=True)# decoding the output
        final_summ_gen= final_summ_gen+summ_generated
    return(final_summ_gen)

In [ ]:
def rouge_score(summaryContent,final_summ_gen):  
    rouge = Rouge()
    result_rouge=rouge.get_scores(summaryContent, final_summ_gen, avg=True)
# Printing the final list
    print(result_rouge)

In [ ]:
# inputs to the model
if __name__ == "__main__":
  FileContent,summaryContent=read_input()
  tokenizer,model=model_load()
  chunks=chunking(FileContent,tokenizer)
  final_summ_gen=sum_generate(chunks,tokenizer,model)
  result_rouge=rouge_score(summaryContent,final_summ_gen)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertLMHeadModel: ['cls.seq_relationship.bias', 'cls.seq_relations

tokenizer.model_max_length : 512
 max tokens excluding the special tokens 510
num_special_tokens_to_add  2
98
maximum tokens in longest sentence: 147
Total no of chunks created 6
Total no of tokens generated by chunks 2675
Total no of token in original text 2665
No of tokens in each  chunk excluding the special token [489, 487, 494, 493, 488, 212]


ValueError: ignored

In [ ]:
print(final_summ_gen)

 Appeal from an Order of the High Court of Bombay (Bavdekar and Chainani JJ.) dated 20th February, 1950, in Criminal Appeal No. 106 of 1950. W.H. King was tenant of a flat on the second floor of a building called "Ganga Vihar", Marine Drive, Bombay, which belongs to a lady named Durgeshwari Devi. The wife of the appellant denied any aiding and abetting. The wife was acquitted, the evidence being insufficient to prove any abetment. The appellant preferred an appeal to the High Court of Bombay but it was summarily dismissed. He asked for a certificate under article 134(1)(c) of the Constitution but this was rejected on 10 4 1950. He applied for special leave to appeal to this Court and it was granted on 3 10 1950. The word "relinquishment" does not occur in the Transfer of Property Act but it is found in many of the Tenancy Acts in various provinces where there are Sec tions which deal with the relinquishment of their holdings by tenants in favour of the landlord by notice given to him i

In [ ]:
print("Given Summary:",summaryContent)

Given Summary: Sub section (1) of sec.
19 of the Bombay Rents, Hotel and Lodging House Rates Control Act, LVI I of 1947, provided that "it shall not be lawful for the tenant or any person acting or purporting to act on behalf of the tenant to claim or receive any sum or any consideration as a condition for the relinquishment of his tenancy of any premises"; and sub sec.
(2) provided that any tenant or person who in contravention of the provisions of sub sec.
(1) receives any sum or consideration shall on conviction be punished with imprisonment and also with fine.
A. who was a tenant of a flat, handed over vacant posses sion the flat to B on receiving "pugree", under a document which recited that A shall have no claim whatever over the flat and that B shall pay the rent directly to the landlord.
A was convicted of an offence under sec.
19 (2).
Held, that there was no "relinquishment" of his tenancy by A, within the meaning of sec.
19 (1) and the conviction could not be sustained.
There

tokenizer.model_max_length : 1024

max tokens excluding the special tokens 1022

num_special_tokens_to_add  2  98

maximum tokens in longest sentence: 137

Total no of chunks created 3

Total no of tokens generated by chunks 2572

Total no of token in original text 2634

No of tokens in each  chunk excluding the special token [999, 1019, 548]



{'rouge-1': {'r': 0.291044776119403, 'p': 0.20855614973262032, 'f': 0.24299064934191253}, 

'rouge-2': {'r': 0.08673469387755102, 'p': 0.049132947976878616, 'f': 0.06273062268923388}, 

'rouge-l': {'r': 0.26119402985074625, 'p': 0.18716577540106952, 'f': 0.21806853096185025}}

# T5 Small - Chunking (Single Document)
T5 Small

In [ ]:
def read_input():
  doc_name = '/content/drive/MyDrive/Legal Data/Indian Legal Abs Ext/dataset/IN-Abs/test-data/judgement/232.txt'
  doc = open(doc_name,'r')
  FileContent = doc.read().strip()
  sum_name ='/content/drive/MyDrive/Legal Data/Indian Legal Abs Ext/dataset/IN-Abs/test-data/summary/232.txt'
  sum = open(sum_name,'r')
  summaryContent = sum.read().strip()
  return(FileContent,summaryContent)


In [ ]:
def model_load():
    checkpoint = "t5-small" #model loaded from Huggingface
    tokenizer = AutoTokenizer.from_pretrained(checkpoint)
    model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)
#Model Details 
    print("tokenizer.model_max_length :",tokenizer.model_max_length )# max token inputted in model 1024
    print(" max tokens excluding the special tokens",tokenizer.max_len_single_sentence )# max tokens for input in model 1022
    print("num_special_tokens_to_add ",tokenizer.num_special_tokens_to_add() ) # special token needed by model 2
    return(tokenizer,model)


In [ ]:
def chunking(FileContent,tokenizer):
    # extract the sentences from the document
    sentences = nltk.tokenize.sent_tokenize(FileContent) #split the data(paragraph) into individual sentences
    print(len(sentences)) #print total no of sentences
    # find the max tokens in the longest sentence
    maxtokencount=max([len(tokenizer.tokenize(sentence)) for sentence in sentences])
    print("maximum tokens in longest sentence:",maxtokencount)

    # initialize
    length = 0
    chunk = ""
    chunks = []
    count = -1
    for sentence in sentences:
        count += 1
        combined_length = len(tokenizer.tokenize(sentence)) + length # add the no. of sentence tokens to the length counter
        #print(combined_length)  #combined length of all the tokens in all sentences 

        if combined_length  <= tokenizer.max_len_single_sentence: # if it doesn't exceed than max_len(1022)
            chunk += sentence + " " # add the sentence to the chunk with white space
            length = combined_length # update the length counter

            # if it is the last sentence
            if count == len(sentences) - 1:
                chunks.append(chunk.strip()) # save the chunk
    
        else:   # if max token limit reaches then save the chunk
            chunks.append(chunk.strip()) # save the chunk
    
        # reset the parameter to add next tokens to next chunk
            length = 0 
            chunk = ""

        # take care of the overflow sentence
            chunk += sentence + " "
            length = len(tokenizer.tokenize(sentence))

            
    print("Total no of chunks created",len(chunks))
    print("Total no of tokens generated by chunks",sum([len(tokenizer(c).input_ids) for c in chunks]))
    print("Total no of token in original text",len(tokenizer(FileContent).input_ids))
    print("No of tokens in each  chunk excluding the special token",[len(tokenizer.tokenize(c)) for c in chunks])
    return (chunks)

In [ ]:
def sum_generate(chunks,tokenizer,model):
    final_summ_gen=""
    inputs = [tokenizer(chunk, return_tensors="pt") for chunk in chunks] #genrating inputs with chunks pt stands for pytorch
    for input in inputs:
        output = model.generate(**input)    #output generated for each chunk
        summ_generated=tokenizer.decode(*output, skip_special_tokens=True)# decoding the output
        final_summ_gen= final_summ_gen+summ_generated
    return(final_summ_gen)

In [ ]:
def rouge_score(summaryContent,final_summ_gen):  
    rouge = Rouge()
    result_rouge=rouge.get_scores(summaryContent, final_summ_gen, avg=True)
# Printing the final list
    print(result_rouge)

In [ ]:
# inputs to the model
if __name__ == "__main__":
  FileContent,summaryContent=read_input()
  tokenizer,model=model_load()
  chunks=chunking(FileContent,tokenizer)
  final_summ_gen=sum_generate(chunks,tokenizer,model)
  result_rouge=rouge_score(summaryContent,final_summ_gen)

/usr/local/lib/python3.8/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-large automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


tokenizer.model_max_length : 512
 max tokens excluding the special tokens 511
num_special_tokens_to_add  1
860
maximum tokens in longest sentence: 198
Total no of chunks created 78


Token indices sequence length is longer than the specified maximum sequence length for this model (37431 > 512). Running this sequence through the model will result in indexing errors


Total no of tokens generated by chunks 37508
Total no of token in original text 37431
No of tokens in each  chunk excluding the special token [470, 502, 505, 511, 500, 500, 471, 494, 503, 472, 465, 483, 507, 511, 491, 492, 502, 475, 474, 410, 473, 473, 456, 478, 487, 456, 475, 447, 497, 472, 488, 502, 506, 509, 476, 492, 504, 435, 509, 484, 501, 463, 499, 506, 486, 511, 505, 492, 492, 505, 502, 421, 479, 488, 494, 468, 480, 498, 494, 480, 452, 490, 494, 499, 445, 482, 423, 506, 498, 449, 496, 371, 499, 502, 509, 503, 445, 246]
{'rouge-1': {'r': 0.42317380352644834, 'p': 0.27495908346972175, 'f': 0.3333333285586932}, 'rouge-2': {'r': 0.16604708798017348, 'p': 0.09824046920821114, 'f': 0.1234454121877156}, 'rouge-l': {'r': 0.380352644836272, 'p': 0.24713584288052373, 'f': 0.29960316982853447}}


In [ ]:
print(final_summ_gen)

of the Court of Appeal, arising out of an order dated 20th February,,,, and the keys of the flat and the motor garage were handed overthe appellant was in search of a partner to carry on his business. thea tion for the relinquishment of his tenancy of anyis not a lawful transaction; it cannot be a unilateral transaction. of a tenancy is equivalent to surrender by the lessee or tenant of his


In [ ]:
print("Given Summary:",summaryContent)

Given Summary: Sub section (1) of sec.
19 of the Bombay Rents, Hotel and Lodging House Rates Control Act, LVI I of 1947, provided that "it shall not be lawful for the tenant or any person acting or purporting to act on behalf of the tenant to claim or receive any sum or any consideration as a condition for the relinquishment of his tenancy of any premises"; and sub sec.
(2) provided that any tenant or person who in contravention of the provisions of sub sec.
(1) receives any sum or consideration shall on conviction be punished with imprisonment and also with fine.
A. who was a tenant of a flat, handed over vacant posses sion the flat to B on receiving "pugree", under a document which recited that A shall have no claim whatever over the flat and that B shall pay the rent directly to the landlord.
A was convicted of an offence under sec.
19 (2).
Held, that there was no "relinquishment" of his tenancy by A, within the meaning of sec.
19 (1) and the conviction could not be sustained.
There

/usr/local/lib/python3.8/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.

For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.

- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.

- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.

- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(

Token indices sequence length is longer than the specified maximum sequence length for this model (2859 > 512). Running this sequence through the model will result in indexing errors

tokenizer.model_max_length : 512

max tokens excluding the special tokens 511

num_special_tokens_to_add  1  98

maximum tokens in longest sentence: 147

Total no of chunks created 6

Total no of tokens generated by chunks 2864

Total no of token in original text 2859

No of tokens in each  chunk excluding the special token [502, 498, 398, 496, 489, 475]

/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.

{'rouge-1': {'r': 0.5686274509803921, 'p': 0.15508021390374332, 'f': 0.24369747562424976}, 

'rouge-2': {'r': 0.31343283582089554, 'p': 0.06069364161849711, 'f': 0.10169491253604121},

'rouge-l': {'r': 0.47058823529411764, 'p': 0.12834224598930483, 'f': 0.2016806689015607}}

# T5 Small - Chunking (Multiple Documents)

In [ ]:
def read_input(i):
  doc_name = f'/content/drive/MyDrive/Legal Data/Indian Legal Abs Ext/dataset/IN-Abs/test-data/judgement/{i}.txt'
  doc = open(doc_name,'r')
  FileContent = doc.read().strip()
  sum_name =f'/content/drive/MyDrive/Legal Data/Indian Legal Abs Ext/dataset/IN-Abs/test-data/summary/{i}.txt'
  sum = open(sum_name,'r')
  summaryContent = sum.read().strip()
  return(FileContent,summaryContent)


In [ ]:
def model_load():
    checkpoint = "t5-small" #model loaded from Huggingface
    tokenizer = AutoTokenizer.from_pretrained(checkpoint)
    model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)
#Model Details 
    #print("tokenizer.model_max_length :",tokenizer.model_max_length )# max token inputted in model 1024
    #print(" max tokens excluding the special tokens",tokenizer.max_len_single_sentence )# max tokens for input in model 1022
    #print("num_special_tokens_to_add ",tokenizer.num_special_tokens_to_add() ) # special token needed by model 2
    return(tokenizer,model)


In [ ]:
def chunking(FileContent,tokenizer):
    # extract the sentences from the document
    sentences = nltk.tokenize.sent_tokenize(FileContent) #split the data(paragraph) into individual sentences
    #print(len(sentences)) #print total no of sentences
    # find the max tokens in the longest sentence
    maxtokencount=max([len(tokenizer.tokenize(sentence)) for sentence in sentences])
    #print("maximum tokens in longest sentence:",maxtokencount)

    # initialize
    length = 0
    chunk = ""
    chunks = []
    count = -1
    for sentence in sentences:
        count += 1
        combined_length = len(tokenizer.tokenize(sentence)) + length # add the no. of sentence tokens to the length counter
        #print(combined_length)  #combined length of all the tokens in all sentences 

        if combined_length  <= tokenizer.max_len_single_sentence: # if it doesn't exceed than max_len(1022)
            chunk += sentence + " " # add the sentence to the chunk with white space
            length = combined_length # update the length counter

            # if it is the last sentence
            if count == len(sentences) - 1:
                chunks.append(chunk.strip()) # save the chunk
    
        else:   # if max token limit reaches then save the chunk
            chunks.append(chunk.strip()) # save the chunk
    
        # reset the parameter to add next tokens to next chunk
            length = 0 
            chunk = ""

        # take care of the overflow sentence
            chunk += sentence + " "
            length = len(tokenizer.tokenize(sentence))

            
    #print("Total no of chunks created",len(chunks))
    #print("Total no of tokens generated by chunks",sum([len(tokenizer(c).input_ids) for c in chunks]))
    #print("Total no of token in original text",len(tokenizer(FileContent).input_ids))
    #print("No of tokens in each  chunk excluding the special token",[len(tokenizer.tokenize(c)) for c in chunks])
    return (chunks)

In [ ]:
def sum_generate(chunks,tokenizer,model):
    final_summ_gen=""
    inputs = [tokenizer(chunk, return_tensors="pt") for chunk in chunks] #genrating inputs with chunks pt stands for pytorch
    for input in inputs:
        output = model.generate(**input)    #output generated for each chunk
        summ_generated=tokenizer.decode(*output, skip_special_tokens=True)# decoding the output
        final_summ_gen= final_summ_gen+summ_generated
    #print(final_summ_gen)
    return(final_summ_gen)

In [ ]:
def rouge_score(summaryContent,final_summ_gen):  
    rouge = Rouge()
    result_rouge=rouge.get_scores(summaryContent, final_summ_gen, avg=True)
# Printing the final list
    return result_rouge

In [ ]:
# inputs to the model
if __name__ == "__main__":
  result_rouge_list = []
  for i in dir_list:
    FileContent,summaryContent=read_input(i)
    tokenizer,model=model_load()
    chunks=chunking(FileContent,tokenizer)
    final_summ_gen=sum_generate(chunks,tokenizer,model)
    result_rouge_list.append(rouge_score(summaryContent,final_summ_gen))

In [ ]:
len(result_rouge_list)

100

In [ ]:
agg_rouge_score = {'rouge-1':{'r':0,'p':0,'f':0},'rouge-2':{'r':0,'p':0,'f':0},
                   'rouge-l':{'r':0,'p':0,'f':0}}

for i in range(len(result_rouge_list)):
  for rouge_type in ['rouge-1','rouge-2','rouge-l']:
    for j in ['r','p','f']:
      agg_rouge_score[rouge_type][j]+=(result_rouge_list[i][rouge_type][j]/len(result_rouge_list))

print(agg_rouge_score)

{'rouge-1': {'r': 0.5340070793670784, 'p': 0.15040051883404526, 'f': 0.2259315977833998}, 'rouge-2': {'r': 0.2703121170814129, 'p': 0.052595345376596325, 'f': 0.08455800139639953}, 'rouge-l': {'r': 0.4836286813566721, 'p': 0.13425102682810058, 'f': 0.20248698625877598}}
